In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd
from openai import OpenAI

from src.embedding_model import embed
from src.utils import populate_everything, return_paragraphs_with_highest_score

In [3]:
question_reference = "Finn forekomster av strategisk bruk av anskaffelser. Eksempler kan inkludere: Offentlig-privat samarbeidsmodeller, partnerskap for innovasjon, dorskning og utvikling"
question_final = (
    "Hvordan omtales strategisk bruk av anskaffelser (i tildelingsbrev og årsrapport)"
)

In [4]:
model = "gpt-4-turbo"
examples_to_retrieve = 20
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

Under definerer vi de referansetekstene vi fikk fra tidligere gjennomgang, siden dette ikke har endret seg.

In [7]:
inn_texts = [
    "strategisk bruk av anskaffelser i offentlig sektor",
    "dokumenter som diskuterer offentlig-privat samarbeid og partnerskap",
    "innovasjon gjennom offentlige anskaffelser og utviklingsprosjekter",
    "forsknings- og utviklingsinitiativer i offentlige anskaffelser",
    "samarbeidsmodeller mellom offentlige institusjoner og private selskaper for teknologisk utvikling",
]

In [8]:
inn_texts_embeddings = [embed(inn_text) for inn_text in inn_texts]
inn_texts_embeddings = np.array(inn_texts_embeddings)
inn_texts_embedding = np.mean(inn_texts_embeddings, axis=0)

/workspaces/DIGDIR-embedding-av-offentlige-dokumenter/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/workspaces/DIGDIR-embedding-av-offentlige-dokumenter/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
df[
    ~(
        df["name"].str.contains("DFØ")
        | df["name"].str.contains("DIGITALISERING")
        | df["title"].str.contains("Digdir")
        | df["title"].str.contains("Digitaliserings")
        | df["title"].str.contains("DFØ")
    )
    & df["concerned_year"].isin([2021, 2022])
]

,id,uuid,type,title,subtitle,original_title,isbn,isbn_printed,issn,redirect_to_id,...,split_paragraphs,lengths,sims,sims_scaled,length_of_split_paragraphs,deviation_scaled_with_length,weighted_mean_sims,pure_mean_sims,top_k_mean,all_mean
416,21902,e1681741-f725-4d98-bf5b-b18d51461c6a,Årsrapport,Årsrapport Spesialenheten for politisaker 2021,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(1, Spesialenheten (;\\nfor politisaker, [[ 0...",2,"[0.151034842945743, 0.11778149143448757]","[0.5386315281981284, 0.700420324237665]","[34, 56]","[0.5679809605939413, 0.6853424442626065]",0.639300,0.619526,0.626662,0.628037
417,21978,88f68804-7aff-4595-a49d-fcd788f05971,Årsrapport,Årsrapport Samfunnet Jan Mayen - Cyberforsvare...,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(1, Årsrapport\n2021\nSamfunnet Jan Mayen\nCy...",54,"[0.14907276721352258, 0.15309798722342016, 0.1...","[0.5481776898939468, 0.5285936338635298, 0.603...","[50, 55, 499, 55, 55, 348, 53, 59, 55, 1202, 5...","[0.5763860994210183, 0.5622398793612006, 0.649...",0.526356,0.526566,0.554292,0.540376
418,21999,2fb05153-61a2-4296-93b3-6beaff8ec70c,Årsrapport,Årsrapport Statped 2021,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(1, 1 • Statped | Årsrapport 2021, [[ 0.04569...",417,"[0.1479275105515938, 0.1353046912540562, 0.117...","[0.5537497506959084, 0.6151640335865216, 0.700...","[29, 32, 560, 342, 179, 82, 26, 35, 424, 682, ...","[0.5806412164547977, 0.6321807789671251, 0.781...",0.566683,0.553340,0.695652,0.599502
419,22047,8efb84ee-adf6-4bf5-9b0d-9d17371c9fdf,Årsrapport,Årsrapport Konfliktrådet 2021,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(2, Sekretariatet for konfliktrådene\nMøllerg...",294,"[0.14509436559895117, 0.13701052019739146, 0.1...","[0.567533958604648, 0.606864599140096, 0.62475...","[90, 509, 95, 63, 88, 420, 59, 1313, 32, 187, ...","[0.5940404612033929, 0.6539227361617013, 0.649...",0.548070,0.531719,0.651516,0.572427
420,22079,51853bd8-9f80-4c3f-8e0c-9d88a6f63a14,Årsrapport,Årsrapport Statens undersøkelseskommisjon for ...,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(1, Statens undersøkelseskommisjon for helse-...",122,"[0.12118680075390975, 0.12222848713408352, 0.1...","[0.6838523432705421, 0.6787841869001368, 0.653...","[68, 129, 315, 127, 98, 36, 67, 38, 73, 42, 35...","[0.6899284443762983, 0.7113791337107683, 0.705...",0.560922,0.580275,0.630568,0.593139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,67831,8edd0a11-77a2-47c4-8104-3668ecea2a8b,Tildelingsbrev,Tildelingsbrev 2022 Stortingets ombudsnemnd fo...,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(1, Stortingets ombudsnemd for forsvaret, [[ ...",6,"[0.1593703680040366, 0.1579436962384798, 0.163...","[0.498076381055073, 0.5050176214655293, 0.4774...","[36, 263, 28, 59, 45, 31]","[0.5352466228178475, 0.5239972067846553, 0.514...",0.506124,0.500896,0.532468,0.517989
1709,75468,f19f1bb3-3179-4505-a01e-422dfca570ce,Tildelingsbrev,Tildelingsbrev Norsk jernbanemuseum 2021,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(1, Norsk jernbanemuseum\nPostboks 491\n2304 ...",38,"[0.1697992572795286, 0.14755243663619522, 0.15...","[0.4473363093561231, 0.5555746120840195, 0.531...","[44, 59, 40, 605, 621, 710, 53, 32, 56, 41, 42...","[0.5034745080817331, 0.5815456768569861, 0.563...",0.557285,0.539866,0.690740,0.589028
1710,75469,d6066e9d-dc6f-466e-9fc2-cb82bba4545f,Tildelingsbrev,Tildelingsbrev Norsk jernbanemuseum 2022,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(1, Norsk Jernbanemuseum - ekstern\nPostboks ...",39,"[0.1668760211897704, 0.15220491912408274, 0.15...","[0.4615588411012681, 0.5329387120588573, 0.543...","[54, 50, 605, 328, 731, 208, 53, 32, 56, 41, 2...","[0.5172230491544303, 0.5763860994210183, 0.569...",0.562484,0.544268,0.700142,0.595483
1716,76616,1ff4524c-4d1e-40f1-88d0-1c4430c9d759,Tildelingsbrev,Tildelingsbrev Kunsthøgskolen i Oslo 2021,NaN,NaN,NaN,NaN,NaN,NaN,...,"[(1, Kunsthøgskolen i Oslo\nPostboks 6853 St O...",84,"[0.17048622881713982, 0.16704087693074887, 0.1...","[0.443993960602316, 0.46075676219370054, 0.740...","[59, 497, 50, 33, 41, 60, 38, 40, 53, 43, 43,

In [9]:
df: pd.DataFrame = pd.read_pickle("df_with_embeddings_float32_filtered50.pkl")
df = df[
    ~(
        df["name"].str.contains("DFØ")
        | df["name"].str.contains("DIGITALISERING")
        | df["title"].str.contains("Digdir")
        | df["title"].str.contains("Digitaliserings")
        | df["title"].str.contains("DFØ")
    )
]

In [10]:
populate_everything(df, inn_texts_embedding)

justeringer for test 2

In [20]:
df = df[df["concerned_year"].isin([2021, 2022])]

justeringer for test 3

In [26]:
df = df[df["type"] == "Årsrapport"]

In [28]:
print(
    return_paragraphs_with_highest_score(
        df, "deviation_scaled_with_length", examples_to_retrieve
    )
)

Virksomhet: NORGES FORSKNINGSRÅD
Dokument: Årsrapport Forskningsrådet 2022
Score: 0.925
tekstsnutt: samarbeid mellom næringsliv og offentlig

Virksomhet: ARKITEKTUR- OG DESIGNHØGSKOLEN I OSLO
Dokument: Årsrapport Arkitektur- og designhøgskolen i Oslo 2022
Score: 0.914
tekstsnutt: Delmål 3.1 AHO skal i løpet av perioden etablere langsiktige forpliktende samarbeid
med sentrale aktører i samfunnet
AHO har i utviklingsavtalens løpetid (2019-2022) gjort noen viktig strategiske valg på
samarbeidspartnere og sektorer. Her følger en kort oppsummering over de ulike tematiske
områdene hvor AHO har bygget langsiktige strategiske samarbeid
D-Box
Våren 2020 lanserte AHO, Handelshøgskolen BI og DOGA et nasjonalt senter for
brukersentrert innovasjon i offentlig sektor og skal gjennom designtekning, tverrfaglig
kompetanser og brukerorientering bidra til å transformere offentlige tjenester og
videreutvikle velferdsnasjonen Norge til et smart, bærekraftig og inkluderende samfunn.
Gjennom dette samarbeid

In [29]:
completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": f"Basert på et sett med teksnutter fra rapporter fra ulike statlige virksomheter som brukeren gir vil jeg at du skal svare på spørsmålet: {question_final}. Pass på at du svarer med grunnlag i tekstsnuttene som brukeren gir, og at du svarer på spørsmålet oppgitt tidligere.",
        },
        {
            "role": "user",
            "content": f"{return_paragraphs_with_highest_score(df, 'deviation_scaled_with_length', examples_to_retrieve)}",
        },
    ],
)
print(completion.choices[0].message.content)

Strategisk bruk av anskaffelser omtales i de nevnte rapportene på ulike måter, med fokus på fremming av samarbeid, bærekraft, innovasjon, og strategisk planlegging i offentlig sektor.

1. Sametinget fremhever bærekraft i offentlige anskaffelser og arbeidet med konkrete kontrakter som støtter samiske kulturelle og utdanningsmessige behov. Det involverer opplæring i offentlige anskaffelser fokusert på mindre innkjøp og deltagelse i fellesavtaler for staten for å sikre større kostnadseffektivitet og bærekraft.

2. Arkitektur- og Designhøgskolen i Oslo setter lys på brukersentrert innovasjon gjennom strategiske langsiktige samarbeid med offentlige og private aktører for å transformere offentlige tjenester og bidra til en bærekraftig samfunnsutvikling.

3. Universitetet i Oslo og NAV illustrerer hvordan de har organisert anskaffelsesprosesser for å inkludere strenge krav for bærekraft, etikk og innovasjon, samt hvordan de fremmer balanserte kontrakter og sosialt ansvar gjennom sine anskaffe